In [1]:
import json
import pandas as pd
import requests
import mysql.connector

# API Request

url = "https://api.sportradar.com/tennis/trial/v3/en/double_competitors_rankings.json?api_key=poXVLPnWV8ZufVVMyW2VkoXbjHjPVpF07nbAVPUL"
headers = {"accept": "application/json"}
response = requests.get(url, headers=headers)

# Load JSON response

data = response.json()

# Extract rankings data

rankings = data.get("rankings", [])  
competitors = []
competitor_rankings = []

for ranking in rankings:
    for competitor_ranking in ranking.get("competitor_rankings", []):
        competitor = competitor_ranking.get("competitor", {})

        # Store competitor details

        competitors.append({
            "competitor_id": competitor.get("id"),
            "name": competitor.get("name",""),
            "country": competitor.get("country",""),
            "country_code": competitor.get("country_code",""),
            "abbreviation": competitor.get("abbreviation","")
        })

        # Store ranking details 

        competitor_rankings.append({
            "ranks": competitor_ranking.get("rank"),
            "movement": competitor_ranking.get("movement"),
            "points": competitor_ranking.get("points"),
            "competitions_played": competitor_ranking.get("competitions_played"),
            "competitor_id": competitor.get("id"),
            
        })

# Convert lists to DataFrames

df_competitors = pd.DataFrame(competitors).drop_duplicates(subset=["competitor_id"])
df_competitor_rankings = pd.DataFrame(competitor_rankings)

# Connect to MySQL Database

mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="NanDha@12345",
    database="sports_radar_api",
    auth_plugin='mysql_native_password'
)
cursor = mydb.cursor()

# Insert Data into Competitors Table

for _, row in df_competitors.iterrows(): 
    sql = """
        INSERT INTO competitors (competitor_id, name, country, country_code, abbreviation) 
        VALUES (%s, %s, %s, %s, %s) 
        ON DUPLICATE KEY UPDATE 
        name = VALUES(name),
        country = VALUES(country),
        country_code = VALUES(country_code),
        abbreviation = VALUES(abbreviation)
    """
    values = (
        row["competitor_id"], 
        row["name"], 
        row["country"], 
        row["country_code"], 
        row["abbreviation"]
    )
    cursor.execute(sql, values)

# Insert Data into Competitor_Rankings Table

for _, row in df_competitor_rankings.iterrows():
    sql = """
    INSERT INTO Competitor_Rankings (ranks, movement, points, competitions_played, competitor_id) 
    VALUES (%s, %s, %s, %s, %s) 
    ON DUPLICATE KEY UPDATE 
    ranks = VALUES(ranks),
    movement = VALUES(movement),
    points = VALUES(points),
    competitions_played = VALUES(competitions_played)
"""

    
    values = (
        row["ranks"],  
        row["movement"], 
        row["points"], 
        row["competitions_played"], 
        row["competitor_id"], 
        
    )
    cursor.execute(sql, values)

# Commit changes and close the connection

mydb.commit()
cursor.close()
mydb.close()
